In [60]:
import pandas as pd
import numpy as np
import tqdm
from collections import defaultdict

import gzip
import pickle

!pip install pymorphy2
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation
import pymorphy2
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from sklearn.semi_supervised import LabelSpreading, LabelPropagation

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import xml.etree.ElementTree as ET
linkrev = 'https://drive.google.com/open?id=10KALb9_A15D0589m1EYiRmUypakFGPaA'
fluff, id = linkrev.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('rest_train.xml')  
tree = ET.parse('rest_train.xml')
root = tree.getroot()
df = pd.DataFrame(columns = ['food_score', 'interior_score', 'service_score', 'review'])
for i in range(19034):
    df.loc[i] = [root[i][1][0].text,root[i][1][1].text,root[i][1][2].text,root[i][2].text]

In [38]:
linkser = "https://drive.google.com/open?id=14o5GJQgZqN03Tc51LzKYb6sa650iOCRw"
fluff, id = linkser.split('=')
downloaded2 = drive.CreateFile({'id':id}) 
downloaded2.GetContentFile('Service_words.csv')
service = pd.read_csv('Service_words.csv', sep = "\\t", names = ['aspect', 'word', 'label'], encoding = 'utf-8')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


In [0]:
linkfood = 'https://drive.google.com/open?id=18bRE2WnURDKHdWoZG-rlaiinLxjIVX_w'
fluff, id = linkfood.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Food_words.csv')
food = pd.read_csv('Food_words.csv', sep = ";",   names = ['aspect', 'word', 'label'], encoding = 'utf-8')

In [0]:
morph = pymorphy2.MorphAnalyzer()
stopWords = stopwords.words('russian')
def preproc(text):
    tokenized = []
    text = re.sub(r'[^\w\s]','',text)
    words = word_tokenize(text)
    for word in words:
        p = morph.parse(word)[0]
        tokenized.append(p.normal_form)
    tokenized = [token for token in tokenized if token not in stopWords\
                and token.strip() not in punctuation]
    return tokenized

In [0]:
#можно не запускать, если уже загружена модель
reviews = df['review'].tolist()
sents = []
for review in reviews:
  tokreview = sent_tokenize(review)
  for sent in tokreview:
    toksent = preproc(sent)
    sents.append(toksent)


In [42]:
modeltrain = Word2Vec(sents, size=200, window=3, min_count=2, iter=10)
modeltrain.save("word2vec.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [43]:
model = Word2Vec.load("word2vec.model")
allwords2 = model.wv.vocab

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
aspectstrainlabels = []
allwords = []
for k, v in allwords2.items():
  allwords.append(model.wv[k])
  if k in service['word'].tolist():
    aspectstrainlabels.append(1)
  elif k in food['word'].tolist():
    aspectstrainlabels.append(0)
  else:
    aspectstrainlabels.append(-1)

In [0]:
labeled_spr = LabelSpreading(kernel='knn', n_neighbors=5, max_iter = 30)
labelsenttest = labeled_spr.fit(allwords, aspectstrainlabels)


In [0]:
wordstest = [i for i in allwords2]

In [0]:
foodwords = []
servicewords = []
for i in range(len(labelsenttest.transduction_)):
  if labelsenttest.transduction_[i] == 0:
    foodwords.append(wordstest[i])
  elif labelsenttest.transduction_[i] == 1:
    servicewords.append(wordstest[i])

In [0]:
foodneg = food[lambda x: x['label'] == 0]['word'].tolist()
foodaff = food[lambda x: x['label'] == 1]['word'].tolist()
servneg = service[lambda x: x['label'] == 0]['word'].tolist()
servaff = service[lambda x: x['label'] == 1]['word'].tolist()

In [0]:
def delw (ml, dell):
  for word in dell:
    if word in ml:
      ml.remove(word)
  return ml
foodwords = delw(foodwords, foodneg)
foodwords = delw(foodwords, foodaff)
servicewords = delw(foodwords, servneg)
servicewords = delw(foodwords, servaff)


In [0]:
foodwordsvecs = []
servicewordsvecs = []
for fword in foodwords:
  foodwordsvecs.append(model.wv[fword])
for sword in servicewords:
  servicewordsvecs.append(model.wv[sword])

In [0]:
foodnegdicti = defaultdict(int)
foodaffdicti = defaultdict(int)
servicenegdicti = defaultdict(int)
serviceaffdicti = defaultdict(int)
for word in foodneg:
  foodnegdicti[word] = 0
for word in foodaff:
  foodaffdicti[word] = 0
for word in servneg:
  servicenegdicti[word] = 0
for word in servaff:
  serviceaffdicti[word] = 0


In [0]:
def dist(x,y):   
    return np.sqrt(np.sum((x-y)**2))

def addtosentiment(dictionary, somewords):
  newword = ''
  distance = 100000000000000000000000000000000
  for word, vec in dictionary.items():
    for food in somewords:
      newdistance = dist(model.wv[word], model.wv[food])
      if vec + newdistance < distance:
        distance = vec + newdistance
        newword = food
  somewords.remove(newword)
  dictionary[newword] = distance

In [0]:
while len(foodnegdicti) < 100:
  addtosentiment(foodnegdicti, foodwords)
while len(foodnegdicti) < 100:
  addtosentiment(foodaffdicti, foodwords)
while len(foodnegdicti) < 100:
  addtosentiment(servicenegdicti, servicewords)
while len(foodnegdicti) < 100:
  addtosentiment(serviceaffdicti, servicewords)
  



In [0]:
def razmetkamaker (text, output):
  f = open(output, "a")
  tokreview = sent_tokenize(text)
  for sent in tokreview:
    toksent = preproc(sent)
    sents.append(toksent)
  for s in range(len(sents)):
    for w in range(len(sents[s])):
      if sents[s][w] in servicewordsneg:
        string = str(s+1) + '\\t' + str(w+1) +'\\t' + 'service' + '\\t' + '0' + '\\n'
        f.write(string)
      elif sents[s][w] in servicewordsaff:
        string = str(s+1) + '\\t' + str(w+1) +'\\t' + 'service' + '\\t' + '1' + '\\n'
        f.write(string)
      elif sents[s][w] in foodwordsneg:
        string = str(s+1) + '\\t' + str(w+1) +'\\t' + 'food' + '\\t' + '0' + '\\n'
        f.write(string)
      elif sents[s][w] in foodwordsaff:
        string = str(s+1) + '\\t' + str(w+1) +'\\t' + 'food' + '\\t' + '1' + '\\n'
        f.write(string)